In [158]:
import pandas as pd

In [159]:
data = pd.read_csv("/home/student/Desktop/Realme Narzo 10 (That White, 128 GB) Reviews.csv")

In [160]:
data.head()

,Rating,Review,Comments
0,4.0,Good choice,After Using 6 Day's My Review. Good Smartphone...
1,5.0,Terrific purchase,Great product in such a good price.Camera qual...
2,5.0,Awesome,After using of 10 days my review--It's a best ...
3,5.0,Excellent,"Very good budget phone, it's good affordable s..."
4,5.0,Worth every penny,"Very good budget phone, everything seems perfe..."


In [161]:
review = data['Review']

In [162]:
import nltk

In [163]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stop_words = stopwords.words('english')

In [164]:
snow = SnowballStemmer('english')

In [165]:
import emoji
import re
from nltk.tokenize import word_tokenize

In [166]:
review = [re.sub("!","",txt) for txt in review]
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
def give_emoji_free_text(comm):
    allchars = [str for str in comm]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in comm.split() if not any(i in str for i in emoji_list)])
    return clean_text

review = [give_emoji_free_text(txt) for txt in review]
review = [give_emoji_free_text(txt) for txt in review]
review = [re.sub("READ MORE|READ MOR|READ MO","",i) for i in review]
review = [re.sub('-','',i) for i in review]
review = [re.sub('\d','',i) for i in review]


    

In [167]:
text =[]
setstop = ['money','purchase','penny','market','product','every']
stopw = (stopwords.words('english'))
stopw.extend(setstop)
stopw = set(stopw)

for message in review:
    message = message.lower().strip()
    words = [snow.stem(word) for word in message.split() if word not in stopw]
    text.append(' '.join(words))

In [186]:
v= []
for val in text:
    val = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
    v.append(" ".join(tokens))
for word in range(len(v)):
    token = word_tokenize(v[word])
    for tkn in token:
        if tkn in stopw:
            token.remove(tkn)
    v[word] = " ".join([str(ele) for ele in token])

In [169]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

In [170]:
count_vect = CountVectorizer(max_features=5000)
bow_data = count_vect.fit_transform(v).toarray()

In [171]:
count_vect.get_feature_names()[:10]

['absolut',
 'amazinnnn',
 'amez',
 'averag',
 'awesom',
 'bad',
 'batteri',
 'beast',
 'best',
 'better']

In [172]:
tf_idf = TfidfVectorizer()
tf_data = tf_idf.fit_transform(v).toarray()
tf_data.shape

(9960, 95)

# Just randomnly placing labels

In [173]:
z = []
import random
for i in range(len(v)):
    c = random.randint(0,1)
    z.append(c)
data['Score'] = z
y = data['Score']

In [174]:
x_train,x_test,y_train,y_test = train_test_split(tf_data,y)

In [175]:
x_train.shape

(7470, 95)

In [176]:
y_test.shape

(2490,)

In [177]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)

/home/student/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [178]:
accuracy_score(y_test,y_pred)

0.4955823293172691

In [179]:
accuracy_score(y_train,lr.predict(x_train))

0.5291834002677376

In [180]:
cm = confusion_matrix(y_test,y_pred)

In [210]:
cm

array([[543, 717],
       [539, 691]])

# Using TextBlob

In [182]:
from textblob import TextBlob
score = []
for i in range(len(v)):
    bl = TextBlob(v[i])
    score.append(bl.sentiment.polarity)
    



In [183]:
for i in range(len(score)):
    if score[i] < 0.0:
        score[i] = 0
    elif score[i] > 0.0:
        score[i] = 1
    elif score[i] == 0.0:
        score[i] = 0
        


In [184]:
data['Score'] = score

In [185]:
data['Score'].value_counts()

0    5050
1    4910
Name: Score, dtype: int64

In [201]:
data.Review = v
vect = TfidfVectorizer()
X=vect.fit_transform(data.Review).toarray()

X_df = pd.DataFrame(X,columns = vect.get_feature_names())


In [197]:

y = data['Score']

In [203]:
X_train,X_test,y_train,y_test = train_test_split(X_df,y,test_size=0.7,random_state=123,stratify=y)
log_reg = LogisticRegression(penalty='l2',C=1.0).fit(X_train,y_train)
print("Trainig set:",log_reg.score(X_train,y_train))
print("Testing set:",log_reg.score(X_test,y_test))

Trainig set: 0.9973226238286479
Testing set: 0.9964142283419392


/home/student/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [204]:
y_pred = log_reg.predict(X_test)

In [205]:
accuracy_score(y_test,y_pred)

0.9964142283419392

In [209]:
accuracy_score(y_train,log_reg.predict(X_train))

0.9973226238286479

In [211]:
cm = confusion_matrix(y_test,y_pred)

In [212]:
cm

array([[3512,   23],
       [   2, 3435]])